**Please only run the following cell if you're running this notebook on Google colab or another cloud hosted notebook server.**

In [0]:
!pip install qiskit

# B92 Quantum Key Distribution (QKD) Protocol

This notebook is a _demonstration_ of the B92 Protocol for QKD using Qiskit. 
B92is a quantum key distribution scheme developed by Charles Bennett in 1992 as an improvement over b92 ([paper]).
The first two pages of the paper are readable and should give you all the necessary information required. 

[paper]: https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.68.3121



In [0]:
from qiskit import BasicAer
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute
from qiskit import IBMQ

from qiskit.tools.monitor import job_monitor
%pylab inline

We will now simulate the effects of eavesdropping on b92. The qubit capabilites of the available backends are listed below:


| Backend | Number of qubits |
|----------|------------------|
|qasm_simulator|24|
|ibmq_qasm_simulator|32|
|ibmqx4|5|
|ibmqx2|5|
|ibmq_16_melbourne|14|

## Choosing bases and encoding states

$0  \rightarrow |0\rangle$

$1  \rightarrow |+\rangle$ 

Bob also generates a binary string and uses the following convention to choose a basis for measurement

$0 \rightarrow$ Hadamard basis

$1 \rightarrow$ Computational basis


In [0]:
num_qubits = 32

alice_state = np.random.randint(2, size=num_qubits)
bob_basis = np.random.randint(2, size=num_qubits)
eve_basis = np.random.randint(2, size=num_qubits)

print(alice_state)
print(bob_basis)
print(eve_basis)

## Creating the circuit

ased on the following result:

$H|0\rangle = |+\rangle$

Our algorithm to construct the circuit is as follows:

1. Whenever Alice wants to encode 1 in a qubit, she applies an $H$ gate to the qubit. To encode 0, no action is needed.

2. She then _sends_ the qubits to Bob (symbolically represented in this circuit using wires)

3. Eve intercepts these qubits and measures them in a basis according to her binary string.

4. Bob measures the qubits according to his binary string. To measure a qubit in the Hadamard basis, he applies an $H$ gate to the corresponding qubit and then performs a mesurement on the computational basis. 

The result of Eve's measurements are stored in the first half of the classical registers and the results of Bob's measurements are stored in the second.


In [0]:
q = QuantumRegister(num_qubits,'q')
c_bob = ClassicalRegister(num_qubits,'c_bob')
c_eve = ClassicalRegister(num_qubits,'c_eve')

b92 = QuantumCircuit(q,c_eve,c_bob,name='b92')

# Alice prepares her qubits
for index, _ in enumerate(alice_state):
    if alice_state[index] == 1:
        b92.h(q[index])
b92.barrier() 

# Eve chooses her bases
for index, _ in enumerate(eve_basis):
    if eve_basis[index] == 0:
        b92.h(q[index])
b92.barrier()  

# Eve measures the qubits
for index, _ in enumerate(eve_basis):
    b92.measure(q[index],c_eve[index])
b92.barrier()        

# Bob chooses his bases
for index, _ in enumerate(bob_basis):
    if bob_basis[index] == 0:
        b92.h(q[index])
b92.barrier()        
   
# Bob measures the received qubits
for index, _ in enumerate(bob_basis):        
    b92.measure(q[index],c_bob[index])
        
b92.barrier()        
    
            
# b92.draw()


### Running it on the IBMQ machines

The following cell is only needed if you want to use the IBM Q Experience devices. Please enter your API token between the quotes. Instructions are given [here](https://github.com/Qiskit/qiskit/blob/master/docs/install.rst#access-ibm-q-devices)
Uncomment the code in the cell to use it. 

In [0]:
# token = ''
# IBMQ.enable_account(token)
# IBMQ.backends()

## Performing the actual experiment

Run the code on the desired backend. Use the cell above to authenticate yourself to the IBMQ network.

In [0]:
backend = BasicAer.get_backend('qasm_simulator')
# backend = IBMQ.get_backend('ibmq_16_melbourne')
# backend = IBMQ.get_backend('ibmq_qasm_simulator')

num_shots = 1024 # Change this to alter the number of times your circuit runs

job = execute(b92,backend,shots=num_shots)

job_monitor(job)

result = job.result()
counts = result.get_counts()


bob_keys_temp = []
eve_keys_temp = []


print("Bob's string\t\tEve's string")
for count in [*counts]:
    bob_key = count[2*num_qubits:num_qubits-1:-1]
    eve_key = count[num_qubits:0:-1]
    bob_keys_temp.append(bob_key)
    eve_keys_temp.append(eve_key)
    print(bob_key + '\t' + eve_key)


In [0]:
alice_keys = []
eve_keys = []
bob_keys = []
for eve_temp_key, bob_temp_key in zip(eve_keys_temp,bob_keys_temp):
    alice_key = ''
    eve_key = ''
    bob_key = ''
    for i in range(num_qubits):
        if bob_temp_key[i] == '1': # Only choose bits where Bob measured a 1
            alice_key += str(alice_state[i])
            eve_key += str(eve_basis[i])
            bob_key += str(bob_basis[i])
    eve_keys.append(eve_key)
    bob_keys.append(bob_key)
    alice_keys.append(alice_key)
    
    
print("Alice's Key\t\tBob's key\t\tEve's key")
for alice_key, eve_key, bob_key in zip(alice_keys, eve_keys, bob_keys):
    print(alice_key + '\t\t' + bob_key + '\t\t' + eve_key)

In [0]:
np.mean([len(key) for key in alice_keys])